# XGBoost Regression

In [77]:
import pandas as pd
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import os
import xgboost as xgb
from pathlib import Path

In [78]:
CITY = "milano"

data_dir = "../../preprocessed/" 
# features_dir = data_dir + "features/features_all/"
separate_features_dir = data_dir + "regression_features/features_separate_cities/"

labels_dir = data_dir + "regression_labels/" 

standardize_features = True

PCA_components = 64


USE_GEO = "GEO"

network_type = "vgg16_4096"

standardize_features = True


cities = ['milano', 'bologna', 'firenze', 'palermo', 'torino']

In [79]:
label_columns = ["hType_mix", "num_intersect", "bld_avg_age", "emp_rat_num",\
				"LUM5_single",	"RNR_nres", "mdist_smallparks", "nig_rat_daily",\
				"nig_rat_daily3", "mdist_nres_daily", "num_community_places", \
				"num_community_places_poi", "avg_block_area", "sphi", \
				"enterprises_empl_size", "pop_rat_num",  \
				"emp_rat_pop", "bld_rat_area", "den_nres_daily",\
				"mdist_parks", "den_nres_non-daily", "mdist_railways",\
				"mdist_highways", "mdist_water", "activity_density"]

In [80]:
land_use = [
"LUM5_single","RNR_nres","mdist_smallparks",
"hType_mix", "nig_rat_daily", "mdist_nres_daily",
"num_community_places", "num_community_places_poi"]


small_blocks = [
"avg_block_area","num_intersect", "sphi"]


age_buildings = [
"bld_avg_age","enterprises_empl_size"]

concentration = [
"pop_rat_num","emp_rat_num","emp_rat_pop",
"bld_rat_area","den_nres_daily","den_nres_non-daily"]

vacuums = [
"mdist_parks", "mdist_railways",
"mdist_highways", "mdist_water"]

## Functions

In [81]:
if network_type == "vgg19":
	features_file = "Italy_6_cities_vgg19_pca"+str(PCA_components)+"_linear_fc_thirdlast_layer.csv"
elif network_type == "resnet50":
	features_file = "Italy_6_cities_resnet_pca"+str(PCA_components)+"_second_last_layer.csv"
elif network_type == "vgg16_4096":
	features_file = "Italy_6_cities_resnet_pca" + str(PCA_components) + "_vgg16_4096.csv"

In [82]:
def get_normalized_labels_features(city_name=CITY):

    df = pd.read_csv(separate_features_dir + \
        features_file.replace(".csv", "_" + city_name + "_labels_features.csv"))

    df["city_image"] = df.\
        apply(lambda x: x.city + "_" + x.imageName, axis = 1)
    
    del df['imageName']
    del df['city']
    del df['index']
    return df

In [87]:
def predict_label_i_KFold(city_name= CITY, label="label_hType_mix"):
    
    kf = KFold(n_splits=5)
    
    
    data3 = data.copy()
    target = data3[label]
    
    if USE_GEO == "GEO":
        features = data3[[c for c in data.columns if ("PCA" in c) or ("centroid" in c)]]
    else:
        features = data3[[c for c in data.columns if "PCA" in c]]
    
    X = features.values
    y = target
    
    
    r2 = []
    rmse = []
    mae = []

    kf.get_n_splits(X, y)
    for train_index, test_index in kf.split(X, y):
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
        param_dist = {'objective' :'reg:squarederror', 'n_estimators':16}
        clf = xgb.XGBRegressor(**param_dist)
        
        clf.fit(X_train, y_train,verbose=False)
        
        predictions = clf.predict(X_test)
        rmse1 = np.sqrt(mean_squared_error(y_test, predictions))
        rmse.append(rmse1)
        
        r21 = r2_score(y_test, predictions)
        r2.append(r21)
        
        mae1 = mean_absolute_error(y_test, predictions)
        mae.append(mae1)

        
    return  ({"MAE": (np.mean(mae), np.std(mae)), \
              "R2": (np.mean(r2), np.std(r2)), "RMSE": (np.mean(rmse), np.std(rmse))},
             {"RMSE": np.mean(rmse), "R2": np.mean(r2), "MAE": np.mean(mae) })

## Read in Data. Chose standardized or not.

In [88]:
if standardize_features:
    data = get_normalized_labels_features()
else:
    data = get_labels_features()
data.head()

,PCA0,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,...,label_emp_rat_pop,label_bld_rat_area,label_den_nres_daily,label_mdist_parks,label_den_nres_non-daily,label_mdist_railways,label_mdist_highways,label_mdist_water,label_activity_density,city_image
0,0.439129,0.613818,0.105353,0.671947,0.584087,0.333288,0.232843,0.007876,0.301698,0.621144,...,0.680503,0.543631,0.715952,0.160769,0.368380,0.907464,0.314818,0.041176,0.659161,milano_S2B_MSIL2A_20181024T102059_N0209_R065_T...
1,0.039299,0.296719,0.725656,0.397496,0.225069,0.061125,0.575850,0.442814,0.378938,0.448510,...,0.550964,0.039828,0.138941,0.410288,0.010717,0.068859,0.286644,0.264237,0.317580,milano_S2B_MSIL2A_20181024T102059_N0209_R065_T...
2,0.537519,0.292588,0.265063,0.141354,0.428936,0.845432,0.354047,0.428587,0.796385,0.574373,...,0.519877,0.866162,0.794297,0.137891,0.807486,0.364515,0.184614,0.620406,0.882345,milano_S2B_MSIL2A_20181024T102059_N0209_R065_T...
3,0.327497,0.338799,0.615740,0.096609,0.275332,0.704205,0.621657,0.604031,0.601702,0.531715,...,0.629492,0.168800,0.120473,0.425658,0.097738,0.278395,0.977022,0.808904,0.325979,milano_S2B_MSIL2A_20181024T102059_N0209_R065_T...
4,0.929802,0.278382,0.297208,0.564842,0.747615,1.000000,0.339163,0.240381,0.396706,0.210041,...,0.222045,0.677125,0.744330,0.417627,0.639340,0.579227,0.027418,0.387328,0.839758,milano_S2B_MSIL2A_20181024T102059_N0209_R065_T...


In [89]:
data.columns

Index(['PCA0', 'PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8',
       'PCA9', 'PCA10', 'PCA11', 'PCA12', 'PCA13', 'PCA14', 'PCA15', 'PCA16',
       'PCA17', 'PCA18', 'PCA19', 'PCA20', 'PCA21', 'PCA22', 'PCA23', 'PCA24',
       'PCA25', 'PCA26', 'PCA27', 'PCA28', 'PCA29', 'PCA30', 'PCA31', 'PCA32',
       'PCA33', 'PCA34', 'PCA35', 'PCA36', 'PCA37', 'PCA38', 'PCA39', 'PCA40',
       'PCA41', 'PCA42', 'PCA43', 'PCA44', 'PCA45', 'PCA46', 'PCA47', 'PCA48',
       'PCA49', 'PCA50', 'PCA51', 'PCA52', 'PCA53', 'PCA54', 'PCA55', 'PCA56',
       'PCA57', 'PCA58', 'PCA59', 'PCA60', 'PCA61', 'PCA62', 'PCA63',
       'centroid_x', 'centroid_y', 'label_bld_avg_age', 'label_hType_mix',
       'label_num_intersect', 'label_LUM5_single', 'label_RNR_nres',
       'label_mdist_smallparks', 'label_nig_rat_daily', 'label_nig_rat_daily3',
       'label_mdist_nres_daily', 'label_num_community_places',
       'label_num_community_places_poi', 'label_avg_block_area', 'label_sphi',
       'labe

## Predict K-Fold

In [90]:
predict_label_i_KFold(CITY, label="label_hType_mix")

({'MAE': (0.17426101585983744, 0.015202318512097635),
  'R2': (0.1687008229765857, 0.12407813786917227),
  'RMSE': (0.21883971520959725, 0.01747855307221262)},
 {'RMSE': 0.21883971520959725,
  'R2': 0.1687008229765857,
  'MAE': 0.17426101585983744})

In [91]:
kfold_SCORES = {}
kfold_SCORES2 = {}
for col in label_columns:
    label = "label_" + col
    (res1, res2) = predict_label_i_KFold(CITY, label)
    kfold_SCORES[label] = res1
    kfold_SCORES2[label] = res2

In [92]:
kfold_SCORES2

{'label_hType_mix': {'RMSE': 0.21883971520959725,
  'R2': 0.1687008229765857,
  'MAE': 0.17426101585983744},
 'label_num_intersect': {'RMSE': 0.19787000887230893,
  'R2': 0.44825266174560846,
  'MAE': 0.15559435717718367},
 'label_bld_avg_age': {'RMSE': 0.17840001788218635,
  'R2': 0.3183740103317815,
  'MAE': 0.14197000503768298},
 'label_emp_rat_num': {'RMSE': 0.16392098639470698,
  'R2': 0.3886554642485013,
  'MAE': 0.12882845733787202},
 'label_LUM5_single': {'RMSE': 0.18410285142372676,
  'R2': 0.17426202560085913,
  'MAE': 0.13719632880958224},
 'label_RNR_nres': {'RMSE': 0.21541242902161378,
  'R2': -0.0028958114260467793,
  'MAE': 0.16522930079234882},
 'label_mdist_smallparks': {'RMSE': 0.16255793626742462,
  'R2': 0.025204761999238313,
  'MAE': 0.1223856700703442},
 'label_nig_rat_daily': {'RMSE': 0.19350924356452123,
  'R2': -0.03338007750087042,
  'MAE': 0.14790884226350506},
 'label_nig_rat_daily3': {'RMSE': 0.21129993769689998,
  'R2': 0.2320914647073235,
  'MAE': 0.12842

In [93]:
res = pd.DataFrame(kfold_SCORES2)

In [94]:
res

,label_hType_mix,label_num_intersect,label_bld_avg_age,label_emp_rat_num,label_LUM5_single,label_RNR_nres,label_mdist_smallparks,label_nig_rat_daily,label_nig_rat_daily3,label_mdist_nres_daily,...,label_pop_rat_num,label_emp_rat_pop,label_bld_rat_area,label_den_nres_daily,label_mdist_parks,label_den_nres_non-daily,label_mdist_railways,label_mdist_highways,label_mdist_water,label_activity_density
MAE,0.174261,0.155594,0.141970,0.128828,0.137196,0.165229,0.122386,0.147909,0.128428,0.140578,...,0.180193,0.131806,0.132623,0.149381,0.148239,0.146283,0.118980,0.125530,0.108628,0.118188
R2,0.168701,0.448253,0.318374,0.388655,0.174262,-0.002896,0.025205,-0.033380,0.232091,0.399993,...,0.203208,0.204028,0.507900,0.471017,0.147625,0.340690,0.538327,0.613880,0.653141,0.429885
RMSE,0.218840,0.197870,0.178400,0.163921,0.184103,0.215412,0.162558,0.193509,0.211300,0.185121,...,0.223214,0.165591,0.178181,0.190872,0.187665,0.191817,0.155064,0.167016,0.146439,0.154912


In [188]:
if standardize_features:
    out_name = '../../results/XGBoost/separate_cities/XGBoost'\
    +str(PCA_components) + '_' + CITY + '_' + network_type \
    + '_' + USE_GEO + '_standardized7s.csv'
else:
    out_name = '../../results/XGBoost/XGBoost' +str(PCA_components)+\
    + '_' + CITY + '_' + network_type + \
    '_' + USE_GEO + '7s.csv'
res.to_csv(out_name, float_format='%.3f')

In [189]:
res_dir = '../../results/XGBoost/separate_cities/' + CITY + '_' + network_type \
         + '_' + str(PCA_components) + '_' + USE_GEO
Path(res_dir).mkdir(parents=True, exist_ok=True)

In [190]:
res

,label_hType_mix,label_num_intersect,label_bld_avg_age,label_emp_rat_num,label_LUM5_single,label_RNR_nres,label_mdist_smallparks,label_nig_rat_daily,label_nig_rat_daily3,label_mdist_nres_daily,...,label_pop_rat_num,label_emp_rat_pop,label_bld_rat_area,label_den_nres_daily,label_mdist_parks,label_den_nres_non-daily,label_mdist_railways,label_mdist_highways,label_mdist_water,label_activity_density
AUC,0.915273,0.968254,0.990476,0.947619,0.876722,0.897934,0.939286,0.915041,0.963265,0.932653,...,0.864,0.725069,0.983333,0.957143,0.848000,0.914286,0.819259,0.925247,0.973669,0.911111
Accuracy,0.841429,0.873077,0.967949,0.830769,0.807510,0.867984,0.872381,0.870563,0.885714,0.836190,...,0.760,0.649802,0.900000,0.839560,0.770000,0.816484,0.753801,0.826462,0.930154,0.820513
Precision,0.838095,0.916667,0.942857,0.873333,0.794643,0.885897,0.885000,0.871667,0.889286,0.872778,...,0.760,0.669286,0.909524,0.865556,0.722611,0.810000,0.781883,0.863258,0.903663,0.808333
Recall,0.861818,0.842857,1.000000,0.842857,0.878788,0.839394,0.857143,0.885455,0.885714,0.785714,...,0.740,0.678788,0.900000,0.823810,0.820000,0.804762,0.722222,0.811538,0.969231,0.871429


In [191]:
land_use_cols = ["label_"+l for l in land_use]
res_land_use = res[land_use_cols]

In [192]:
small_blocks_cols = ["label_"+l for l in small_blocks]
res_small_blocks = res[small_blocks_cols]

In [193]:
age_buildings_cols = ["label_"+l for l in age_buildings]
res_age_buildings = res[age_buildings_cols]

In [194]:
concentration_cols = ["label_"+l for l in concentration]
res_concentration = res[concentration_cols]

In [195]:
vacuums_cols = ["label_"+l for l in vacuums]
res_vacuums = res[vacuums_cols]

In [196]:
for out_cat_name in ["land_use", "small_blocks", \
                     "age_buildings", "concentration",
                    "vacuums"]:
    eval("res_" + out_cat_name).to_csv(res_dir + "/res_" + out_cat_name+\
                                       ".csv", float_format='%.3f')

# Latex Table for the AUC Results for all cities

In [7]:
cities = ['milano', 'bologna', 'firenze', 'palermo', 'torino']

In [14]:
def predict_label_i_KFold_city(city_name, label):
    
    data = get_normalized_labels_features(city_name=city_name)
    
    data2 = data.copy()
    
    try:
        data3 = data2[data2[label] != 1 ].copy()
    except Exception as e:
        print (e)
        return None
    
    target = data3[label].apply(lambda x: int(x) if x == 0 else 1)
    
    if USE_GEO == "GEO":
        features = data3[[c for c in data.columns if ("PCA" in c) or ("centroid" in c)]]
    else:
        features = data3[[c for c in data.columns if "PCA" in c]]
    
    X = features.values
    y = target
    
    rus = RandomUnderSampler(random_state=1)
    X_resampled, y_resampled = rus.fit_sample(X, y)
    
    acc = []
    auc = []
    P = []
    R = []
    
#     print (y_resampled)
#     print (sum(y_resampled),len(y_resampled))
    kf = StratifiedKFold(n_splits=5)

    kf.get_n_splits(X_resampled, y_resampled)
    for train_index, test_index in kf.split(X_resampled, y_resampled):
        
        X_train, X_test = X_resampled[train_index], X_resampled[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]
    
        param_dist = {'objective':'binary:logistic', 'n_estimators':16}
        clf = xgb.XGBModel(**param_dist)
        
        clf.fit(X_train, y_train,
#                 eval_metric='auc',
                verbose=False)
        
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions.round())
        precision=precision_score(y_test, predictions.round())
        recall=recall_score(y_test, predictions.round())
        roc=roc_auc_score(y_test,predictions)
        
#         print (predictions)
#         print (predictions.round())
#         print (y_test)
#         print (sum(y_test), len(y_test))
        
        acc.append(accuracy)
        auc.append(roc)
        P.append(precision)
        R.append(recall)
        
    return  np.mean(auc)

In [16]:
city_label_res = {}

for city_name in cities:
    city_label_res[city_name] = {}
    for col in land_use:
        label = "label_" + col
        res_cl = predict_label_i_KFold_city(city_name, label)
        city_label_res[city_name][col] = res_cl
df_city_label_res = pd.DataFrame(city_label_res)
df_city_label_res.to_latex("../../results/writing_tables/AUC_per_land_use_var_per_city.tex", \
                           float_format="{:.2f}".format )
  

for city_name in cities:
    city_label_res[city_name] = {}
    for col in small_blocks:
        label = "label_" + col
        res_cl = predict_label_i_KFold_city(city_name, label)
        city_label_res[city_name][col] = res_cl
df_city_label_res = pd.DataFrame(city_label_res)
df_city_label_res.to_latex("../../results/writing_tables/AUC_per_small_blocks_var_per_city.tex", \
                           float_format="{:.2f}".format )    
   
for city_name in cities:
    city_label_res[city_name] = {}
    for col in age_buildings:
        label = "label_" + col
        res_cl = predict_label_i_KFold_city(city_name, label)
        city_label_res[city_name][col] = res_cl
df_city_label_res = pd.DataFrame(city_label_res)
df_city_label_res.to_latex("../../results/writing_tables/AUC_per_age_buildings_var_per_city.tex", \
                           float_format="{:.2f}".format )      
    
for city_name in cities:
    city_label_res[city_name] = {}
    for col in concentration:
        label = "label_" + col
        res_cl = predict_label_i_KFold_city(city_name, label)
        city_label_res[city_name][col] = res_cl
df_city_label_res = pd.DataFrame(city_label_res)
df_city_label_res.to_latex("../../results/writing_tables/AUC_per_concentration_var_per_city.tex", \
                           float_format="{:.2f}".format )    
  
for city_name in cities:
    city_label_res[city_name] = {}
    for col in vacuums:
        label = "label_" + col
        res_cl = predict_label_i_KFold_city(city_name, label)
        city_label_res[city_name][col] = res_cl
df_city_label_res = pd.DataFrame(city_label_res)
df_city_label_res.to_latex("../../results/writing_tables/AUC_per_vacuums_var_per_city.tex", \
                           float_format="{:.2f}".format )      

'label_bld_rat_area'
'label_bld_rat_area'
